In [1]:
import pandas as pd
import time
from concurrent.futures import ProcessPoolExecutor
import dask.dataframe as dd
start_full = time.time()


/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)


In [2]:
candidate_path = '/home/it/Desktop/RUN/RoundData/Round5/Candidate.csv'
choice_path = '/home/it/Desktop/RUN/RoundData/Round5/Choice.csv'
seats_path = '/home/it/Desktop/RUN/RoundData/Round5/Seats.csv'

candidate = pd.read_csv(candidate_path)
choice = pd.read_csv(choice_path)
choice_filtered = choice[choice['Validity'] != 'N']
seat = pd.read_csv(seats_path)

In [3]:

start_ti = time.time()
df = pd.merge(choice_filtered, candidate, on="RollNo")
df.to_csv('/home/it/Desktop/RUN/Final/Round5Final/candidate_choice_merge_R5.csv', index=False)
end_ti = time.time()
print(f"time taken to merge candidate and choice and save as csv- {end_ti- start_ti}")

time taken to merge candidate and choice and save as csv- 377.1188476085663


In [4]:
institutes={}
for index, row in seat.iterrows():
    if row['Quota']=='OS' or row['Quota']=='HS':
        if not row['InstCode'] in institutes.keys():
            institutes[row['InstCode']]=[row['Stcode1'],row['Stcode2'],row['Stcode3'],row['Stcode4'],row['Stcode5'],row['Stcode6'],row['Stcode7'],row['Stcode8'],row['Stcode9'],row['Stcode10']]

In [5]:
del df, candidate, choice, seat

In [6]:
def determine_virtual_categories(row):
    if row['Cat'] == 'BC':
        return ['OP', 'BC']
    elif row['Cat'] == 'OP':
        return ['OP']
    elif row['Cat'] == 'EW':
        return ['OP', 'EW']
    elif row['Cat'] == 'SC':
        return ['OP', 'SC']
    elif row['Cat'] == 'ST':
        return ['OP', 'ST']
    else:
        return []

def determine_virtual_sub_categories(row):
    if row['SubCat'] == 'NO':
        return ['NO']
    elif row['SubCat'] == 'PH':
        return ['NO', 'PH']
    else:
        return []

def determine_virtual_gender(row):
    if row['Gender'] == 'F':
        return ['F', 'B']
    elif row['Gender'] == 'B':
        return ['B']
    else:
        return []

def determine_virtual_state_quota(row):
    if row['InstQuota'] == 'AI':
        return ['AI']

    if row['InstCode'] == 209 and row['StateCode']=='GO':
        return ['GO']    
    
    if row['InstCode'] == 225 and row['StateCode']=='JK':
        return ['JK']    
    
    if row['InstCode'] == 225 and row['StateCode']=='LA':
        return ['LA']    
    state_codes = institutes[row['InstCode']]
    if row['StateCode'] in state_codes:
        if row['InstQuota'] == 'OH':
            return ['HS']
        else:
            return ['AI', 'HS']
    else:
        if row['InstQuota'] == 'OH':
            return ['OS']
        else:
            return ['AI']


def process_chunk(chunk):
    chunk = chunk[
        # (chunk['RStatus'] == 'RP') & 
        # (chunk['Withdraw'] == 'N') & 
        # (chunk['Validity'] != 'N') &    
        (
            (chunk['Decision'] == 'FL') | 
            ((chunk['Decision'] == 'FR') & (chunk['ChoiceNo'] >= chunk['AllotedOpt'])) | 
            ((chunk['Decision'] == 'SL') & 
            ((chunk['ChoiceNo'] >= chunk['AllotedOpt']) | ((chunk['ChoiceNo'] < chunk['AllotedOpt']) & chunk['Instcd'].notna() & (chunk['Instcd'].fillna(-1).astype(int) == chunk['InstCode']))))
        )
    ]
    chunk['Cat_V'] = chunk.apply(determine_virtual_categories, axis=1)
    chunk = chunk.explode('Cat_V')
    chunk['SubCat_V'] = chunk.apply(determine_virtual_sub_categories, axis=1)
    chunk = chunk.explode('SubCat_V')
    chunk['Gender_V'] = chunk.apply(determine_virtual_gender, axis=1)
    chunk = chunk.explode('Gender_V')
    chunk['State_quota_V'] = chunk.apply(determine_virtual_state_quota, axis=1)
    chunk = chunk.explode('State_quota_V')
    return chunk

start_time_loading_merging = time.time()

chunks = pd.read_csv(r'candidate_choice_merge_R5.csv', chunksize=70000)

processed_chunks = []

with ProcessPoolExecutor() as executor:
    for processed_chunk in executor.map(process_chunk, chunks):
        processed_chunks.append(processed_chunk)

merged_df = pd.concat(processed_chunks, ignore_index=True)

end_time_loading_merging = time.time()
print(f"Time taken for loading tables and merging and running virtulization: {end_time_loading_merging - start_time_loading_merging:.2f} seconds")


print(f"Now Sorting:")
merged_df.sort_values(by=['RollNo', 'ChoiceNo'], inplace=True)
end_time_sorting = time.time()

print(f"Time taken for Sorting: {end_time_sorting - end_time_loading_merging:.2f} seconds")


print(f"Now creating Virtual count")
merged_df['ChoiceNo_V'] = merged_df.groupby(['RollNo']).cumcount() + 1
end_time_V = time.time()
print(f"Time taken for creating Virtual count: {end_time_V- end_time_sorting:.2f} seconds")


# Ensure Adv_DS_V column is empty initially
merged_df['Adv_DS_V'] = None

# Identify RollNos with Adv_DS == 1.0
adv_ds_rollnos = merged_df[merged_df['Adv_DS'] == 1.0]['RollNo'].unique()

ds_df = merged_df[merged_df['RollNo'].isin(adv_ds_rollnos)]
normal_df = merged_df[~merged_df['RollNo'].isin(adv_ds_rollnos)]

# Create new rows for DS RollNos at the end of each choice
new_rows = []
for rollno in adv_ds_rollnos:
    unique_choices = ds_df[ds_df['RollNo'] == rollno]['ChoiceNo'].unique()
    for choice in unique_choices:
        choice_rows = ds_df[(ds_df['RollNo'] == rollno) & (ds_df['ChoiceNo'] == choice)]
        if choice_rows.iloc[0]['InstCode'] < 200:  # Only add DS rows for IITs
            last_virtual_choice_no = choice_rows['ChoiceNo_V'].max() + 1
            last_row = choice_rows.iloc[-1]
            new_row = last_row.copy()
            new_row['ChoiceNo_V'] = last_virtual_choice_no
            new_row['Adv_DS_V'] = 1
            new_rows.append(new_row)

new_rows_df = pd.DataFrame(new_rows)

# Append new rows to the DS DataFrame
ds_final_df = pd.concat([ds_df, new_rows_df], ignore_index=True)

# Recalculate VirtualChoiceNumber to account for new DS rows
ds_final_df.sort_values(by=['RollNo', 'ChoiceNo'], inplace=True)
ds_final_df['ChoiceNo_V'] = ds_final_df.groupby(['RollNo']).cumcount() + 1

# Select columns to reorder them as needed
reordered_columns = ['RollNo', 'InstCode', 'Brcode', 'ChoiceNo', 'ChoiceNo_V', 'Cat_V', 'SubCat_V', 'Gender_V', 'State_quota_V', 'Validity', 'Adv_DS_V','Flag']

normal_df = normal_df[reordered_columns]
ds_final_df = ds_final_df[reordered_columns]

print(f"Now creating csv file")
normal_df.to_csv('/home/it/Desktop/RUN/Final/Round5Final/Normal_Virtualized_Choices_R5.csv', index=False)
ds_final_df.to_csv('/home/it/Desktop/RUN/Final/Round5Final/DS_Virtualized_Choices_R5.csv', index=False)


print(f"Time taken for creating csv file: {time.time() - end_time_V:.2f} seconds")

/usr/lib/python3.12/concurrent/futures/process.py:197: DtypeWarning: Columns (66) have mixed types. Specify dtype option on import or set low_memory=False.
  chunk = tuple(itertools.islice(it, chunksize))
/usr/lib/python3.12/concurrent/futures/process.py:197: DtypeWarning: Columns (66) have mixed types. Specify dtype option on import or set low_memory=False.
  chunk = tuple(itertools.islice(it, chunksize))
/usr/lib/python3.12/concurrent/futures/process.py:197: DtypeWarning: Columns (66) have mixed types. Specify dtype option on import or set low_memory=False.
  chunk = tuple(itertools.islice(it, chunksize))
/tmp/ipykernel_1470284/3903085642.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['Cat_V'] = chunk.apply(determine_virtual_categor

Time taken for loading tables and merging and running virtulization: 206.86 seconds
Now Sorting:
Time taken for Sorting: 34.31 seconds
Now creating Virtual count
Time taken for creating Virtual count: 2.22 seconds
Now creating csv file
Time taken for creating csv file: 156.26 seconds


In [7]:
end_full = time.time()
total_time= end_full -start_full
print(f"Total time taken  - {total_time}")
print(f" In minutes - {total_time/60}")

Total time taken  - 794.4750998020172
 In minutes - 13.241251663366954
